In [7]:
import medmnist
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers, models
import torch
from torchvision import transforms


In [5]:
dataTest = medmnist.BreastMNIST(split="test", download=True, size=64)
x_test = np.array([i[0] for i in dataTest])
y_test = np.array([i[1][0] for i in dataTest])

In [8]:
# Para Keras (modelo hiperafinado)
x_test_keras = x_test / 255.0
x_test_keras = np.expand_dims(x_test_keras, axis=-1)  # (N, 64, 64, 1)

# Para PyTorch (Transfer Learning)


transform_torch = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

x_test_torch = torch.stack([transform_torch(img.astype(np.uint8)) for img in x_test])  # (N, 3, 224, 224)

In [ ]:
#Función usada en el otro ipynb
def modeloCustomizable(
    input_shape=(224, 224, 1), 
    num_filtros=[10, 20], 
    tamano_kernel=(5, 5), 
    capas_densas=[50], 
    dropout_rate=[0.5, 0],
    dropout_rate_final=0.5,
):
    if len(num_filtros) != len(dropout_rate):
        raise ValueError("""
            El largo de num_filtros y el de dropout_rate representan la cantidad de 
            capas convolucionales, por lo que estas listas deben tener el mismo largo.
        """)
    capas_conv = [layers.Input(shape=input_shape)]
    for i in range(len(num_filtros)):
        capas_conv.append(layers.Conv2D(num_filtros[i], tamano_kernel, activation='relu'))
        capas_conv.append(layers.MaxPooling2D(pool_size=(2, 2)))
        if dropout_rate[i] > 0:
            capas_conv.append(layers.Dropout(dropout_rate[i]))

    capas_dens = [layers.Flatten()]
    for i in capas_densas:
        capas_dens.append(layers.Dense(i, activation='relu'))
    if dropout_rate_final > 0:
        capas_dens.append(layers.Dropout(dropout_rate_final))
    capas_dens.append(layers.Dense(1, activation='sigmoid'))

    capas_total = capas_conv + capas_dens
    model = models.Sequential(capas_total)
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

mejor_modelo_base = modeloCustomizable(
    input_shape=(64, 64, 1),
    num_filtros=[30, 30],
    tamano_kernel=(9, 9),
    capas_densas=[100]*3,
    dropout_rate=[0, 0],
    dropout_rate_final=0
)

In [10]:
# Keras
# Suponiendo que tienes el modelo cargado como mejor_modelo_base
pred_keras = mejor_modelo_base.predict(x_test_keras).ravel()

# PyTorch
def get_pytorch_preds(model, x_tensor):
    model.eval()
    preds = []
    with torch.no_grad():
        for img in x_tensor:
            img = img.unsqueeze(0)  # (1, 3, 224, 224)
            output = model(img)
            preds.append(output.cpu().numpy().ravel()[0])
    return np.array(preds)

# Suponiendo que tienes los modelos TL cargados como resnet_model, densenet_model, vgg_model
pred_resnet = get_pytorch_preds(resnet_model, x_test_torch)
pred_densenet = get_pytorch_preds(densenet_model, x_test_torch)
pred_vgg = get_pytorch_preds(vgg_model, x_test_torch)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


NameError: name 'resnet_model' is not defined